# Preparación de Datos - Dataset mHealt

## 🎯 Objetivo
Este notebook prepara los datos del dataset mHealth para el análisis posterior.

---

## 1. Configuración del entorno

In [3]:



import pandas as pd
import numpy as np
import urllib.request
import zipfile
import os
import matplotlib.pyplot as plt

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 2. Crear estructura de directorios

In [4]:
# Crear directorios necesarios
directorios = ['data/raw', 'data/processed', 'images/final', 'images/exploratory']

for directorio in directorios:
    if not os.path.exists(directorio):
        os.makedirs(directorio)
        print(f"✅ Directorio creado: {directorio}")
    else:
        print(f"📁 Directorio ya existe: {directorio}")

✅ Directorio creado: data/raw
✅ Directorio creado: data/processed
✅ Directorio creado: images/final
✅ Directorio creado: images/exploratory


## 3. Descargar dataset mHealth

### 📥 Descarga automática desde UCI Repository

In [5]:
def descargar_mhealth():
    """
    Descarga el dataset mHealth desde UCI Repository
    """
    url = "https://archive.ics.uci.edu/static/public/319/mhealth+dataset.zip"
    
    # Verificar si ya existe
    if os.path.exists('data/raw/mhealth_complete.txt'):
        print("✅ Dataset ya existe localmente")
        return True
    
    try:
        print("📥 Descargando dataset mHealth...")
        print("⏳ Esto puede tomar unos minutos...")
        
        # Descargar
        urllib.request.urlretrieve(url, 'data/raw/mhealth_dataset.zip')
        print("✅ Descarga completada")
        
        # Extraer
        print("📂 Extrayendo archivos...")
        with zipfile.ZipFile('data/raw/mhealth_dataset.zip', 'r') as zip_ref:
            # Buscar archivo principal
            archivos = zip_ref.namelist()
            archivo_principal = None
            
            # Buscar el archivo más grande (probablemente los datos)
            for archivo in archivos:
                if archivo.endswith('.log') or 'subject' in archivo.lower() or archivo.endswith('.txt'):
                    zip_ref.extract(archivo, 'data/raw/')
                    old_path = os.path.join('data/raw/', archivo)
                    new_path = 'data/raw/mhealth_complete.txt'
                    if os.path.exists(old_path):
                        os.rename(old_path, new_path)
                        archivo_principal = archivo
                        break
            
            if not archivo_principal:
                # Si no encuentra archivo específico, extraer todo
                zip_ref.extractall('data/raw/')
                print("📂 Todos los archivos extraídos - revisar data/raw/")
        
        print("✅ Dataset extraído exitosamente")
        
        # Limpiar archivo zip
        os.remove('data/raw/mhealth_dataset.zip')
        return True
        
    except Exception as e:
        print(f"❌ Error al descargar: {e}")
        print("💡 Solución: Descarga manual desde:")
        print("   https://archive.ics.uci.edu/static/public/319/mhealth+dataset.zip")
        return False

# Ejecutar descarga
exito = descargar_mhealth()

📥 Descargando dataset mHealth...
⏳ Esto puede tomar unos minutos...
✅ Descarga completada
📂 Extrayendo archivos...
✅ Dataset extraído exitosamente


### 🔍 **Nota importante**
Si la descarga automática no funciona, puedes descargar manualmente:
1. Ve a: https://archive.ics.uci.edu/static/public/319/mhealth+dataset.zip
2. Extrae en la carpeta `data/raw/`
3. Renombra el archivo principal a `mhealth_complete.txt`

## 4. Verificar datos descargados

In [6]:
# Verificar archivos en data/raw/
print("📁 Archivos en data/raw/:")
if os.path.exists('data/raw/'):
    archivos = os.listdir('data/raw/')
    for archivo in archivos:
        ruta = os.path.join('data/raw/', archivo)
        if os.path.isfile(ruta):
            size = os.path.getsize(ruta)
            print(f"  📄 {archivo}: {size:,} bytes ({size/(1024*1024):.1f} MB)")

📁 Archivos en data/raw/:
  📄 mhealth_complete.txt: 30,019,680 bytes (28.6 MB)


In [7]:
# Verificar el archivo principal
archivo_principal = None

# Buscar el archivo de datos más grande
if os.path.exists('data/raw/'):
    archivos = [f for f in os.listdir('data/raw/') if os.path.isfile(os.path.join('data/raw/', f))]
    if archivos:
        # Encontrar el archivo más grande (probablemente los datos)
        archivo_mas_grande = max(archivos, key=lambda f: os.path.getsize(os.path.join('data/raw/', f)))
        ruta_completa = os.path.join('data/raw/', archivo_mas_grande)
        
        # Si no existe mhealth_complete.txt, renombrar el más grande
        if not os.path.exists('data/raw/mhealth_complete.txt') and archivo_mas_grande != 'mhealth_complete.txt':
            os.rename(ruta_completa, 'data/raw/mhealth_complete.txt')
            print(f"📝 Renombrado: {archivo_mas_grande} → mhealth_complete.txt")

# Verificar archivo final
if os.path.exists('data/raw/mhealth_complete.txt'):
    size = os.path.getsize('data/raw/mhealth_complete.txt')
    print(f"✅ Archivo principal: {size:,} bytes ({size/(1024*1024):.1f} MB)")
    
    # Leer primeras líneas
    with open('data/raw/mhealth_complete.txt', 'r') as f:
        primeras_lineas = [f.readline().strip() for _ in range(3)]
    
    print("\n📋 Primeras líneas del dataset:")
    for i, linea in enumerate(primeras_lineas, 1):
        print(f"Línea {i}: {linea[:80]}...")
else:
    print("❌ Error: Archivo principal no encontrado")

✅ Archivo principal: 30,019,680 bytes (28.6 MB)

📋 Primeras líneas del dataset:
Línea 1: -9.8184	0.009971	0.29563	0.0041863	0.0041863	2.1849	-9.6967	0.63077	0.1039	-0.84...
Línea 2: -9.8489	0.52404	0.37348	0.0041863	0.016745	2.3876	-9.508	0.68389	0.085343	-0.838...
Línea 3: -9.6602	0.18185	0.43742	0.016745	0.037677	2.4086	-9.5674	0.68113	0.085343	-0.838...


## 5. Crear muestra para desarrollo

Para facilitar el desarrollo y testing, crearemos una muestra más pequeña del dataset.

In [8]:
def crear_muestra(input_file, output_file, n_filas=10000):
    """
    Crea una muestra del dataset para desarrollo rápido
    """
    try:
        print(f"📊 Creando muestra de {n_filas:,} filas...")
        
        # Definir nombres de columnas según documentación mHealth
        columnas = [
            'accel_chest_x', 'accel_chest_y', 'accel_chest_z',      # Acelerómetro pecho
            'ecg_1', 'ecg_2',                                        # ECG
            'accel_ankle_x', 'accel_ankle_y', 'accel_ankle_z',       # Acelerómetro tobillo
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',          # Giroscopio tobillo
            'magnet_ankle_x', 'magnet_ankle_y', 'magnet_ankle_z',    # Magnetómetro tobillo
            'accel_arm_x', 'accel_arm_y', 'accel_arm_z',             # Acelerómetro brazo
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',                # Giroscopio brazo
            'magnet_arm_x', 'magnet_arm_y', 'magnet_arm_z',          # Magnetómetro brazo
            'actividad'                                               # Actividad (0-12)
        ]
        
        # Leer muestra del dataset
        df_muestra = pd.read_csv(input_file, sep='\t', names=columnas, nrows=n_filas)
        
        # Información básica
        print(f"📋 Dimensiones de la muestra: {df_muestra.shape}")
        print(f"📊 Actividades encontradas: {sorted(df_muestra['actividad'].unique())}")
        
        # Guardar muestra
        df_muestra.to_csv(output_file, index=False)
        
        print(f"✅ Muestra guardada en: {output_file}")
        return True
        
    except Exception as e:
        print(f"❌ Error creando muestra: {e}")
        return False

# Crear muestra para desarrollo
if os.path.exists('data/raw/mhealth_complete.txt'):
    exito_muestra = crear_muestra('data/raw/mhealth_complete.txt', 'data/raw/sample_mhealth.csv', 15000)
else:
    print("❌ No se puede crear muestra: archivo principal no encontrado")

📊 Creando muestra de 15,000 filas...
📋 Dimensiones de la muestra: (15000, 24)
📊 Actividades encontradas: [np.int64(0), np.int64(1), np.int64(2)]
✅ Muestra guardada en: data/raw/sample_mhealth.csv


## 📝 Resumen y próximos pasos

### ✅ Tareas completadas:
1. Descarga del dataset mHealth
2. Verificación de la integridad de los datos  
3. Creación de muestra para desarrollo
4. Preparación de estructura de directorios

### 🎯 Próximo paso:
Ejecutar `02_analisis_exploratorio.ipynb` para realizar el análisis detallado.

In [9]:
print("=" * 60)
print("📋 RESUMEN DE PREPARACIÓN DE DATOS")
print("=" * 60)

# Verificar archivos finales
archivos_verificar = [
    ('data/raw/mhealth_complete.txt', 'Dataset completo'),
    ('data/raw/sample_mhealth.csv', 'Muestra para desarrollo'),
]

todos_listos = True

for archivo, descripcion in archivos_verificar:
    if os.path.exists(archivo):
        size = os.path.getsize(archivo)
        print(f"✅ {descripcion}: {size:,} bytes ({size/(1024*1024):.1f} MB)")
    else:
        print(f"❌ {descripcion}: NO ENCONTRADO")
        todos_listos = False

if todos_listos:
    print(f"\n🎉 ¡Preparación completada exitosamente!")
    print(f"🎯 Listo para ejecutar: 02_analisis_exploratorio.ipynb")
else:
    print(f"\n  Algunos archivos faltan. Revisar pasos anteriores.")

print("=" * 60)

📋 RESUMEN DE PREPARACIÓN DE DATOS
✅ Dataset completo: 30,019,680 bytes (28.6 MB)
✅ Muestra para desarrollo: 2,857,713 bytes (2.7 MB)

🎉 ¡Preparación completada exitosamente!
🎯 Listo para ejecutar: 02_analisis_exploratorio.ipynb
